In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data = tf.keras.utils.audio_dataset_from_directory('./data/data/', batch_size=32) 

Found 10380 files belonging to 90 classes.


In [ ]:
dataset_to_numpy = list(data.as_numpy_iterator())
shape = tf.shape(dataset_to_numpy)
print(shape)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(6000, 1)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dense(90, activation='softmax'))

In [5]:
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [ ]:
data = data.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(16)
data = data.prefetch(8)
train = data.take(36)
test = data.skip(36).take(15)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
checkpoint = tf.keras.callbacks.ModelCheckpoint('/tmp/checkpoint', monitor='val_accuracy', save_best_only=True)
history = model.fit(data, epochs=20, callbacks=[callback, checkpoint], verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 5998, 16)          64        
                                                                 
 conv1d_1 (Conv1D)           (None, 5996, 16)          784       
                                                                 
 flatten (Flatten)           (None, 95936)             0         
                                                                 
 dense (Dense)               (None, 24)                2302488   
                                                                 
 dense_1 (Dense)             (None, 90)                2250      
                                                                 
Total params: 2,305,586
Trainable params: 2,305,586
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
